# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_df = pd.read_csv('weatherdata.csv')
city_df.dropna(inplace = True) 
city_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
hums = city_df["Humidity"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hums, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# filtering - > boolean series 
# access from dataframe

In [ ]:
#desirable values
temp_lower_limit = city_df["Max Temp"] >= 28
temp_upper_limit = city_df["Max Temp"] <= 32
wind_speed_limit = city_df["Wind Speed"] <=10
wind_speed_limit = city_df["Wind Speed"] >=5
cloudiness_limit = city_df["Cloudiness"] <=15

hotel_df = city_df.loc[temp_lower_limit & temp_upper_limit & wind_speed_limit & cloudiness_limit]
hotel_df.loc[:,"Hotel Name"] = "nurgle"
hotel_df

In [ ]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for ind in hotel_df.index:
    print(ind)
    print()
    
    target_coordinates = str(hotel_df['Lat'][ind])+", "+str(hotel_df['Lng'][ind])
    target_radius = 5000
    target_type = "lodging"
    target_search = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        #"keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    try:
        hotel_df.loc[ind,"Hotel Name"] = response.json()["results"][0]["name"]
    except:
        hotel_df.loc[ind,"Hotel Name"] = float("NaN")
        print("no hotels in radius")
hotel_df


In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"<br><b>Hotel</b>:</br> {hotel_df['Hotel Name'][ind]}, <br><b>City:</br></b> {hotel_df['City'][ind]}, <br><b>Country:</br></b> {hotel_df['Country'][ind]}" for ind in hotel_df.index])
fig.add_layer(markers)

fig.add_layer(heat_layer)
fig

# Display figure
